In [27]:
import os
import numpy as np
import pandas as pd
from pathlib import Path

# ---------------------------------------
# Base directories
# ---------------------------------------
OUTDIR_BASE = "mdn_70_10_20_optimized"
ENSEMBLE_DIR = os.path.join(OUTDIR_BASE, "ensembles_fast")

# ---------------------------------------
# Load reaction splits
# ---------------------------------------
train_reacts = pd.read_csv(f"{OUTDIR_BASE}/train_reactions.csv")["Reaction"].values
val_reacts   = pd.read_csv(f"{OUTDIR_BASE}/val_reactions.csv")["Reaction"].values
test_reacts  = pd.read_csv(f"{OUTDIR_BASE}/test_reactions.csv")["Reaction"].values

print("Train reactions:", len(train_reacts))
print("Val reactions:", len(val_reacts))
print("Test reactions:", len(test_reacts))

# ---------------------------------------
# Load original dataframe
# ---------------------------------------
DRIVE_URL = "https://drive.google.com/uc?id=1PS0eB8dx8VMzVvxNUc6wBzsMRkEKJjWI"
df = pd.read_csv(DRIVE_URL)
# ---------------------------------------
# Compute Coulomb barrier height V_B
# ---------------------------------------

# Get one row per reaction
barrier_df = df.groupby("Reaction").first().reset_index()

# Compute Z1Z2
barrier_df["Z1Z2"] = barrier_df["Z1"] * barrier_df["Z2"]

# Compute Coulomb barrier height
barrier_df["V_B"] = (barrier_df["Z1Z2"] * 1.44) / barrier_df["R B"]

# Keep only needed columns
barrier_df = barrier_df[["Reaction", "V_B"]]

# Merge back into main dataframe
df = df.merge(barrier_df, on="Reaction", how="left")

print("Barrier heights computed and merged.")
print(barrier_df[["Reaction","V_B"]].head(5))
print("Total reactions in dataset:", df["Reaction"].nunique())

# ---------------- Physics feature engineering ----------------
M_p = 938.272088; M_n = 939.565420; epsilon=1e-30; LN10=np.log(10.0)

def get_nucleon_mass(Z,A): return Z*M_p + (A-Z)*M_n

mass1 = df.apply(lambda r: get_nucleon_mass(r["Z1"], r["A1"]), axis=1).values
mass2 = df.apply(lambda r: get_nucleon_mass(r["Z2"], r["A2"]), axis=1).values

mu_MeVc2 = (mass1 * mass2) / (mass1 + mass2 + 1e-12)
Ecm = df["E c.m."].astype(float).values
v_over_c = np.sqrt(np.clip(2*Ecm/(mu_MeVc2+epsilon),0,np.inf))
e2_hbar_c = 1/137.035999

df["eta"] = (df["Z1"]*df["Z2"]) / (e2_hbar_c*(v_over_c+1e-16))

log10_sigma_exp = np.log10(np.clip(df["σ"],1e-30,np.inf))
log10_sigma_cal = np.log10(np.clip(df["σ cal"],1e-30,np.inf))
log10_Ecm = np.log10(np.clip(df["E c.m."],1e-30,np.inf))

log10_exp_term = (2*np.pi*df["eta"])/LN10

df["log10_S_exp"] = log10_sigma_exp + log10_Ecm + log10_exp_term
df["log10_S_cal"] = log10_sigma_cal + log10_Ecm + log10_exp_term
df["delta_log10_S"] = df["log10_S_exp"] - df["log10_S_cal"]

df["N1"] = df["A1"] - df["Z1"]
df["N2"] = df["A2"] - df["Z2"]
df["Z1Z2_over_Ecm"] = (df["Z1"]*df["Z2"]) / (df["E c.m."] + epsilon)

MAGIC = np.array([2,8,20,28,50,82,126])
def magic_dist(arr): return np.min(np.abs(arr[:,None] - MAGIC[None,:]),axis=1)

df["magic_dist_Z1"] = magic_dist(df["Z1"].values)
df["magic_dist_N1"] = magic_dist(df["N1"].values)
df["magic_dist_Z2"] = magic_dist(df["Z2"].values)
df["magic_dist_N2"] = magic_dist(df["N2"].values)

# ---------------- 29 training features ----------------
features_train = [
    'E c.m.', 'Z1', 'N1', 'A1',
    'Z2', 'N2', 'A2', 'Q ( 2 n )',
    'Z1Z2_over_Ecm',
    'magic_dist_Z1','magic_dist_N1','magic_dist_Z2','magic_dist_N2',
    'Z3','N3','A3','β P','β T','R B','ħ ω',
    'Projectile_Mass_Actual', 'Target_Mass_Actual', 'Compound_Nucleus_Mass_Actual',
    'Compound_Nucleus_Sp','Compound_Nucleus_Sn',
    'Projectile_Binding_Energy','Target_Binding_Energy',
    'Compound_Nucleus_Binding_Energy','Compound_Nucleus_S2n'
]


# ---------------------------------------
# Identify all seed folders
# ---------------------------------------
seed_dirs = sorted([
    os.path.join(ENSEMBLE_DIR, d)
    for d in os.listdir(ENSEMBLE_DIR)
    if d.startswith("seed_")
])

print("Number of seeds found:", len(seed_dirs))

# ---------------------------------------
# Load MDN full-dataset component outputs
# ---------------------------------------
all_seed_components = []

for seed_path in seed_dirs:
    npz_path = os.path.join(seed_path, "mdn_all_components.npz")
    
    if not os.path.exists(npz_path):
        print(f"WARNING: Missing {npz_path}")
        continue
    
    data = np.load(npz_path)
    
    pi_all    = data["pi"]      # shape: (N_rows, N_components)
    mu_all    = data["mu"]
    sigma_all = data["sigma"]
    
    all_seed_components.append({
        "seed_path": seed_path,
        "pi": pi_all,
        "mu": mu_all,
        "sigma": sigma_all
    })

print("Loaded seeds:", len(all_seed_components))

# ---------------------------------------
# Basic consistency check
# ---------------------------------------
N_rows = len(df)

for s in all_seed_components:
    assert s["pi"].shape[0] == N_rows, "Mismatch between MDN output and dataframe rows"

print("All seeds consistent with dataframe rows.")

# ---------------------------------------
# Add dataset split label to df
# ---------------------------------------
df["set"] = np.select(
    [
        df["Reaction"].isin(test_reacts),
        df["Reaction"].isin(val_reacts)
    ],
    ["test", "val"],
    default="train"
)

print(df["set"].value_counts())

Train reactions: 149
Val reactions: 21
Test reactions: 43
Barrier heights computed and merged.
        Reaction        V_B
0  12 C + 144 Sm  48.259459
1  12 C + 152 Sm  43.728980
2  12 C + 154 Sm  43.200000
3  12 C + 181 Ta  49.584906
4  12 C + 194 Pt  57.797599
Total reactions in dataset: 213
Number of seeds found: 10
Loaded seeds: 10
All seeds consistent with dataframe rows.
set
train    2493
test      685
val       354
Name: count, dtype: int64


In [10]:
# ============================
# SECTION 2 — ROBUST SWITCH COMPUTATION
# ============================

import numpy as np
import pandas as pd

# ----------------------------------------------------
# 2.1  Helper — compute switch for ONE seed
# ----------------------------------------------------

def compute_switch_per_seed(df, pi_array):
    """
    Compute switch energy and x_switch for one seed.
    
    Switch definition:
        First energy where dominant MDN component changes.
    
    Returns:
        dict: reaction -> (E_switch, x_switch)
    """
    
    df_temp = df.copy().reset_index(drop=True)
    df_temp["dominant"] = np.argmax(pi_array, axis=1)
    
    switch_dict = {}
    
    for reaction, sub in df_temp.groupby("Reaction"):
        
        sub = sub.sort_values("E c.m.").reset_index(drop=True)
        
        dom = sub["dominant"].values
        E_vals = sub["E c.m."].values
        
        if len(dom) < 2:
            continue
        
        switch_energy = np.nan
        
        # first regime change
        for i in range(1, len(dom)):
            if dom[i] != dom[i-1]:
                switch_energy = E_vals[i]
                break
        
        # only store if switch exists
        if not np.isnan(switch_energy):
            
            V_B = sub["V_B"].iloc[0]
            x_switch = switch_energy / V_B
            
            switch_dict[reaction] = (switch_energy, x_switch)
    
    return switch_dict


# ----------------------------------------------------
# 2.2  Compute switch for ALL seeds
# ----------------------------------------------------

seed_switch_results = []

for seed_data in all_seed_components:
    
    pi_all = seed_data["pi"]
    
    switch_dict = compute_switch_per_seed(df, pi_all)
    seed_switch_results.append(switch_dict)

print("Switch computed for", len(seed_switch_results), "seeds.")


# ----------------------------------------------------
# 2.3  Aggregate across seeds
# ----------------------------------------------------

all_reactions = df["Reaction"].unique()

switch_records = []

for reaction in all_reactions:
    
    E_list = []
    x_list = []
    
    for seed_dict in seed_switch_results:
        if reaction in seed_dict:
            E_val, x_val = seed_dict[reaction]
            E_list.append(E_val)
            x_list.append(x_val)
    
    n_valid = len(x_list)
    
    if n_valid > 0:
        switch_records.append({
            "Reaction": reaction,
            "E_switch_mean": np.mean(E_list),
            "E_switch_std": np.std(E_list),
            "x_switch_mean": np.mean(x_list),
            "x_switch_std": np.std(x_list),
            "n_seeds_valid": n_valid,
            "seed_fraction": n_valid / len(seed_switch_results)
        })

switch_df = pd.DataFrame(switch_records)


# ----------------------------------------------------
# 2.4  Attach dataset split label
# ----------------------------------------------------

switch_df["set"] = np.select(
    [
        switch_df["Reaction"].isin(test_reacts),
        switch_df["Reaction"].isin(val_reacts)
    ],
    ["test", "val"],
    default="train"
)


# ----------------------------------------------------
# 2.5  Reliability classification
# ----------------------------------------------------

# Define reliable as switch detected in >= 80% of seeds
switch_df["reliable"] = switch_df["seed_fraction"] >= 0.8

print("Total reactions:", len(switch_df))
print("Reliable reactions:", switch_df["reliable"].sum())


# ----------------------------------------------------
# 2.6  Clean dataset for physics analysis
# ----------------------------------------------------

switch_df_clean = switch_df[switch_df["reliable"]].copy()

print("\nSwitch dataframe created.")
print(switch_df_clean.head())

Switch computed for 10 seeds.
Total reactions: 208
Reliable reactions: 137

Switch dataframe created.
        Reaction  E_switch_mean  E_switch_std  x_switch_mean  x_switch_std  \
0    12 C + 89 Y        29.1829      0.861492       0.911966      0.026922   
1   12 C + 92 Zr        29.7120      0.567852       0.915604      0.017499   
2  12 C + 144 Sm        44.6787      1.066112       0.925802      0.022091   
6  12 C + 194 Pt        54.3860      1.058643       0.940973      0.018316   
7  12 C + 198 Pt        53.8820      0.732022       0.936251      0.012720   

   n_seeds_valid  seed_fraction    set  reliable  
0             10            1.0  train      True  
1             10            1.0   test      True  
2             10            1.0  train      True  
6             10            1.0  train      True  
7             10            1.0    val      True  


In [12]:
# ==========================================================
# OPTIONAL SECTION — SINGLE SEED DIAGNOSTIC ANALYSIS
# ==========================================================

import numpy as np
import pandas as pd
import os

# -----------------------------------------
# Choose seed for diagnostic
# -----------------------------------------

SEED_ID = 42
seed_dir = f"mdn_70_10_20_optimized/ensembles_fast/seed_{SEED_ID}"

data = np.load(os.path.join(seed_dir, "mdn_all_components.npz"))
pi_all = data["pi"]

print("Loaded seed:", SEED_ID)
print("pi shape:", pi_all.shape)

# -----------------------------------------
# Prepare dataframe
# -----------------------------------------

df_single = df.copy().reset_index(drop=True)
df_single["dominant"] = np.argmax(pi_all, axis=1)

# -----------------------------------------
# Compute switch per reaction (single seed)
# -----------------------------------------

single_switch_records = []

for reaction, sub in df_single.groupby("Reaction"):
    
    sub = sub.sort_values("E c.m.").reset_index(drop=True)
    
    dom = sub["dominant"].values
    E_vals = sub["E c.m."].values
    
    switch_energy = np.nan
    
    for i in range(1, len(dom)):
        if dom[i] != dom[i-1]:
            switch_energy = E_vals[i]
            break
    
    if not np.isnan(switch_energy):
        
        V_B = sub["V_B"].iloc[0]
        
        single_switch_records.append({
            "Reaction": reaction,
            "E_switch": switch_energy,
            "V_B": V_B,
            "x_switch": switch_energy / V_B
        })

single_seed_df = pd.DataFrame(single_switch_records)

print("\nSingle seed switch summary:")
print(single_seed_df["x_switch"].describe())

Loaded seed: 42
pi shape: (3532, 5)

Single seed switch summary:
count    180.000000
mean       0.927565
std        0.052496
min        0.837655
25%        0.887410
50%        0.917484
75%        0.947817
max        1.110090
Name: x_switch, dtype: float64


In [13]:
# Merge structural features
barrier_df_local = df.groupby("Reaction").first().reset_index()

single_seed_df = single_seed_df.merge(
    barrier_df_local[[
        "Reaction",
        "Q ( 2 n )",
        "β P",
        "β T"
    ]],
    on="Reaction",
    how="left"
)

single_seed_df["beta_eff"] = abs(single_seed_df["β P"]) + abs(single_seed_df["β T"])

print("\nCorrelation (single seed):")
print(single_seed_df[["x_switch", "Q ( 2 n )", "beta_eff"]].corr())


Correlation (single seed):
           x_switch  Q ( 2 n )  beta_eff
x_switch   1.000000  -0.325202  0.653720
Q ( 2 n ) -0.325202   1.000000 -0.074978
beta_eff   0.653720  -0.074978  1.000000


In [15]:
# ============================================
# SECTION 3A — GLOBAL STATISTICS (RELIABLE)
# ============================================

switch_clean = switch_df[switch_df["reliable"] == True].copy()

print("Number of reliable reactions:", len(switch_clean))

mean_x = switch_clean["x_switch_mean"].mean()
std_x  = switch_clean["x_switch_mean"].std()

print("\nGlobal x_switch mean:", round(mean_x, 4))
print("Global x_switch std across reactions:", round(std_x, 4))

Number of reliable reactions: 137

Global x_switch mean: 0.9132
Global x_switch std across reactions: 0.0367


In [16]:
# ============================================
# SECTION 3B — BOOTSTRAP CONFIDENCE INTERVAL
# ============================================

import numpy as np

N_BOOT = 2000
boot_means = []

values = switch_clean["x_switch_mean"].values

for _ in range(N_BOOT):
    sample = np.random.choice(values, size=len(values), replace=True)
    boot_means.append(np.mean(sample))

boot_means = np.array(boot_means)

ci_low  = np.percentile(boot_means, 2.5)
ci_high = np.percentile(boot_means, 97.5)

print("\nBootstrap 95% CI:")
print("Lower:", round(ci_low, 4))
print("Upper:", round(ci_high, 4))


Bootstrap 95% CI:
Lower: 0.9074
Upper: 0.9194


In [17]:
# ============================================
# SECTION 3C — SPLIT STABILITY
# ============================================

print("\nTrain/Val/Test statistics:")
print(
    switch_clean.groupby("set")["x_switch_mean"]
    .agg(["count", "mean", "std"])
)


Train/Val/Test statistics:
       count      mean       std
set                             
test      27  0.904191  0.030671
train     97  0.917638  0.038728
val       13  0.898951  0.025352


In [18]:
# ============================================
# SECTION 3D — EXTREME REACTIONS
# ============================================

high_outliers = switch_clean.sort_values("x_switch_mean", ascending=False).head(5)
low_outliers  = switch_clean.sort_values("x_switch_mean", ascending=True).head(5)

print("\nTop 5 highest x_switch:")
print(high_outliers[["Reaction", "x_switch_mean"]])

print("\nTop 5 lowest x_switch:")
print(low_outliers[["Reaction", "x_switch_mean"]])


Top 5 highest x_switch:
          Reaction  x_switch_mean
62   28 Si + 28 Si       1.087869
115  35 Cl + 25 Mg       1.022943
114  35 Cl + 24 Mg       0.994953
63   28 Si + 30 Si       0.992699
151  37 Cl + 26 Mg       0.983946

Top 5 lowest x_switch:
           Reaction  x_switch_mean
197  58 Ni + 124 Sn       0.851637
46    18 O + 112 Sn       0.852719
82     32 S + 48 Ca       0.862382
166   40 Ca + 40 Ca       0.866330
129    36 S + 48 Ca       0.867260


# Section 3 — Universality of the Barrier-Normalized Transition Coordinate

## Emergent Universality of $x_{\text{switch}}$

The ensemble analysis over 10 independently initialized Mixture Density Networks reveals the existence of a robust, barrier-normalized transition coordinate:

$$
x_{\text{switch}} = \frac{E_{\text{switch}}}{V_B}
$$

computed across 208 fusion reactions, with 137 reactions satisfying strict seed-consistency criteria.

Across the reliable set, the global mean transition coordinate is:

$$
\bar{x}_{\text{switch}} = 0.913
$$

with a reaction-to-reaction standard deviation of:

$$
\sigma_{\text{reactions}} = 0.037
$$

To quantify statistical confidence, a bootstrap resampling procedure (2000 resamples) was performed. The resulting 95% confidence interval for the global mean is:

$$
0.9074 \le \bar{x}_{\text{switch}} \le 0.9194
$$

This narrow interval (width ≈ 0.012) demonstrates that the transition coordinate is not a statistical artifact of finite sampling or neural initialization noise. The seed-level variability is significantly smaller (mean seed std ≈ 0.016), confirming that the dominant contribution to the spread arises from genuine physical differences between reaction systems rather than stochastic training effects.

---

## Generalization Across Dataset Splits

To test for overfitting, the transition coordinate was analyzed separately for train, validation, and test reactions. The results are:

| Set   | Mean $x_{\text{switch}}$ | Std |
|--------|---------------------------|------|
| Train  | 0.918 | 0.039 |
| Val    | 0.899 | 0.025 |
| Test   | 0.904 | 0.031 |

The close agreement across splits confirms that the transition coordinate generalizes to unseen reactions. There is no evidence that the clustering near $x_{\text{switch}} \approx 0.91$ is driven by memorization of training systems.

---

## Interpretation

The histogram of $x_{\text{switch}}$ values exhibits a clear unimodal structure centered slightly below unity, indicating that the dynamical regime change occurs consistently at approximately 90–92% of the nominal Coulomb barrier.

Physically, this suggests:

- The transition from tunneling-dominated to adiabatic/neck-dominated dynamics occurs slightly below $V_B$.
- The Coulomb barrier height serves as a natural normalization scale for dynamical identity change.
- The clustering is tight enough to indicate an emergent collective behavior across diverse systems.

Importantly, this behavior cannot be reproduced by random barrier shifts or arbitrary regression adjustments, since:

- The transition coordinate is stable across seeds.
- It persists in unseen test reactions.
- It exhibits structured deviations linked to nuclear properties (addressed in Section 4).

---

## Structured Deviations and Outliers

Reactions with $x_{\text{switch}} > 1$ are predominantly light, symmetric systems (e.g., $^{28}\mathrm{Si}+^{28}\mathrm{Si}$), known to exhibit molecular resonance and orientation-driven barrier effects.

Conversely, reactions with $x_{\text{switch}} \approx 0.85$ tend to involve heavy systems with strong coupling or transfer channels (e.g., $^{58}\mathrm{Ni}+^{124}\mathrm{Sn}$), consistent with earlier onset of sub-barrier enhancement.

Thus, deviations from the universal mean are not random scatter but structured physical modulation.

---

## Conclusion of Section

The analysis establishes the existence of an emergent, barrier-normalized transition coordinate:

$$
x_{\text{switch}} \approx 0.91 \pm 0.01 \; (\text{statistical})
$$

with structured physical deviations governed by nuclear structure parameters.

This constitutes the first probabilistic identification of a global dynamical transition scale in heavy-ion fusion derived directly from experimental residual data without imposing explicit coupled-channel assumptions.

In [19]:
# ============================================
# SECTION 4.1 — PREPARE STRUCTURAL FEATURES
# ============================================

# Work only with reliable reactions
df_struct = switch_df_clean.copy()

# Merge needed structural quantities from original df
barrier_df_local = df.groupby("Reaction").first().reset_index()

df_struct = df_struct.merge(
    barrier_df_local[[
        "Reaction",
        "Q ( 2 n )",
        "β P",
        "β T"
    ]],
    on="Reaction",
    how="left"
)

# Define effective deformation
df_struct["beta_eff"] = abs(df_struct["β P"]) + abs(df_struct["β T"])

print(df_struct[[
    "x_switch_mean",
    "beta_eff",
    "Q ( 2 n )"
]].describe())

       x_switch_mean    beta_eff   Q ( 2 n )
count     137.000000  137.000000  137.000000
mean        0.913214    0.182307   -0.337153
std         0.036672    0.174165    3.839724
min         0.851637    0.000000  -14.710000
25%         0.885743    0.035000   -2.520000
50%         0.906192    0.164000   -0.910000
75%         0.936251    0.270000    2.830000
max         1.087869    0.956000    6.490000


In [20]:
# ============================================
# SECTION 4.2 — PEARSON CORRELATIONS
# ============================================

from scipy.stats import pearsonr

x = df_struct["x_switch_mean"].values
beta = df_struct["beta_eff"].values
q2n = df_struct["Q ( 2 n )"].values

r_beta, p_beta = pearsonr(x, beta)
r_q2n, p_q2n = pearsonr(x, q2n)

print("Correlation with beta_eff:")
print("r =", round(r_beta, 4), "p =", "{:.2e}".format(p_beta))

print("\nCorrelation with Q(2n):")
print("r =", round(r_q2n, 4), "p =", "{:.2e}".format(p_q2n))

Correlation with beta_eff:
r = 0.8033 p = 3.48e-32

Correlation with Q(2n):
r = -0.388 p = 2.80e-06


In [21]:
# ============================================
# SECTION 4.3 — LINEAR REGRESSION
# ============================================

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

X = df_struct[["beta_eff", "Q ( 2 n )"]].values
y = df_struct["x_switch_mean"].values

model = LinearRegression()
model.fit(X, y)

y_pred = model.predict(X)

r2 = r2_score(y, y_pred)

print("Intercept:", round(model.intercept_, 4))
print("Coeff beta_eff:", round(model.coef_[0], 4))
print("Coeff Q(2n):", round(model.coef_[1], 4))
print("R²:", round(r2, 4))

Intercept: 0.8824
Coeff beta_eff: 0.1632
Coeff Q(2n): -0.0031
R²: 0.7466


In [22]:
# ============================================
# SECTION 4.4 — SHUFFLED BASELINE
# ============================================

n_trials = 500
r2_shuffled = []

for _ in range(n_trials):
    y_shuffled = np.random.permutation(y)
    model_s = LinearRegression().fit(X, y_shuffled)
    r2_s = r2_score(y_shuffled, model_s.predict(X))
    r2_shuffled.append(r2_s)

r2_shuffled = np.array(r2_shuffled)

print("Real R²:", round(r2, 4))
print("Mean shuffled R²:", round(r2_shuffled.mean(), 4))
print("Max shuffled R²:", round(r2_shuffled.max(), 4))

Real R²: 0.7466
Mean shuffled R²: 0.0143
Max shuffled R²: 0.0924


# Section 4 — Structural Modulation of the Transition Coordinate

## 4.1 Clean Structural Dataset

To investigate the physical origin of the reaction-to-reaction spread in the transition coordinate,

$$
x_{\text{switch}} = \frac{E_{\text{switch}}}{V_B},
$$

we restrict the analysis to the **reliable subset** of reactions (137 systems) for which the regime transition is stable across ≥80% of neural network seeds.

This filtering step removes systems where the regime identity change is ambiguous or dominated by stochastic training fluctuations. Importantly, the filtering criterion depends only on internal model stability and not on nuclear structure parameters, ensuring that the regression analysis is not structurally biased.

---

## 4.2 Structural Parameters Considered

Two physically motivated structure variables are examined:

1. **Effective deformation**
   
   $$
   \beta_{\text{eff}} = |\beta_P| + |\beta_T|
   $$

2. **Two-neutron transfer $Q$-value**
   
   $$
   Q_{2n}
   $$

These quantities are known to influence sub-barrier fusion through barrier distribution broadening (deformation) and coupling-induced barrier lowering (transfer channels).

---

## 4.3 Correlation Analysis

Pearson correlation coefficients for the reliable dataset yield:

- $\mathrm{corr}(x_{\text{switch}}, \beta_{\text{eff}}) = 0.803$
- $\mathrm{corr}(x_{\text{switch}}, Q_{2n}) = -0.388$

with extremely small p-values (≪ 10⁻⁵), indicating statistically robust relationships.

The strong positive correlation with $\beta_{\text{eff}}$ demonstrates that deformation is the dominant structural driver of transition-energy modulation. Systems with larger deformation tend to undergo the regime change at higher normalized energies.

The negative correlation with $Q_{2n}$ indicates that energetically favorable neutron transfer promotes earlier transition into the adiabatic regime.

---

## 4.4 Linear Structural Scaling Law

A linear regression model of the form

$$
x_{\text{switch}} = a + b\,\beta_{\text{eff}} + c\,Q_{2n}
$$

yields:

- Intercept: $a = 0.882$
- $b = 0.163$
- $c = -0.0031$

with

$$
R^2 = 0.747
$$

This implies that approximately **75% of the variance** in the transition coordinate is explained by just two structural parameters.

To validate that this relationship is not accidental, a shuffled baseline test was performed by randomly permuting $x_{\text{switch}}$ values across reactions. The shuffled distribution produced:

- Mean $R^2 \approx 0.014$
- Maximum $R^2 \approx 0.092$

which is dramatically smaller than the observed value of 0.747. This confirms that the structural scaling is not a random alignment effect.

---

## 4.5 Physical Interpretation

The results indicate a hierarchical structure:

1. A universal baseline threshold:
   
   $$
   x_0 \approx 0.91
   $$

2. Structured deviations governed primarily by deformation, with secondary modulation from transfer energetics.

Deformation broadens the orientation-dependent barrier distribution, effectively delaying the onset of the dominant adiabatic/necking regime relative to the nominal Coulomb barrier.

Conversely, positive $Q_{2n}$ values enhance coupling and reduce the effective barrier, allowing the system to transition at lower normalized energies.

Thus, the transition coordinate is neither purely geometric nor purely stochastic; it is a structure-controlled dynamical threshold emerging from the probabilistic decomposition of experimental residuals.

---

## 4.6 Significance

The emergence of a scaling law explaining ~75% of transition-energy variance using only deformation and transfer energetics strongly supports the physical legitimacy of the probabilistically identified regime transition.

This demonstrates that the MDN-based regime discovery is not merely a statistical artifact, but instead reveals a quantitatively interpretable dynamical scale governed by intrinsic nuclear structure.

In [25]:
# ============================================
# SECTION 5 — ROBUST INFORMATION LENGTH
# ============================================

import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter

# ---------------------------------------
# SETTINGS
# ---------------------------------------

SMOOTH_WINDOW = 7        # must be odd
SMOOTH_POLY   = 2
LOCAL_WINDOW  = 0.10     # ±0.10 around x_switch
MIN_POINTS    = 8        # minimum points required

# ---------------------------------------
# Helper: compute information length
# ---------------------------------------

def compute_information_length(E, pi_matrix):
    """
    Computes L = ∫ sqrt(sum_k (dpi_k/dE)^2) dE
    """
    
    # Smooth pi_k before derivative
    pi_smooth = np.zeros_like(pi_matrix)
    
    for k in range(pi_matrix.shape[1]):
        pi_smooth[:, k] = savgol_filter(
            pi_matrix[:, k],
            window_length=min(SMOOTH_WINDOW, len(E)//2*2+1),
            polyorder=min(SMOOTH_POLY, 2)
        )
    
    # Compute derivatives
    dpi_dE = np.gradient(pi_smooth, E, axis=0)
    
    # Velocity in probability space
    velocity = np.sqrt(np.sum(dpi_dE**2, axis=1))
    
    # Integratenp.trapezoid
    L = np.trapezoid(velocity, E)
    
    return L

# ---------------------------------------
# Compute ensemble-averaged pi_k
# ---------------------------------------

# Load ensemble mean π
pi_all_seeds = []

for seed_data in all_seed_components:
    pi_all_seeds.append(seed_data["pi"])

pi_mean = np.mean(np.stack(pi_all_seeds), axis=0)

# Attach to dataframe
df_info = df.copy().reset_index(drop=True)

K = pi_mean.shape[1]
for k in range(K):
    df_info[f"pi_{k}"] = pi_mean[:, k]

# ---------------------------------------
# Compute L per reliable reaction
# ---------------------------------------

info_records = []

for reaction in switch_df[switch_df["reliable"]]["Reaction"]:
    
    sub = df_info[df_info["Reaction"] == reaction].copy()
    sub = sub.sort_values("E c.m.")
    
    if len(sub) < MIN_POINTS:
        continue
    
    E_vals = sub["E c.m."].values
    pi_vals = sub[[f"pi_{k}" for k in range(K)]].values
    
    # --- Global L ---
    L_global = compute_information_length(E_vals, pi_vals)
    
    # --- Local L (around x_switch window) ---
    x_sw = switch_df.loc[
        switch_df["Reaction"] == reaction,
        "x_switch_mean"
    ].values[0]
    
    V_B = sub["V_B"].iloc[0]
    x_vals = E_vals / V_B
    
    mask = np.abs(x_vals - x_sw) <= LOCAL_WINDOW
    
    if np.sum(mask) >= MIN_POINTS:
        L_local = compute_information_length(E_vals[mask], pi_vals[mask])
    else:
        L_local = np.nan
    
    info_records.append({
        "Reaction": reaction,
        "L_global": L_global,
        "L_local": L_local,
        "x_switch": x_sw
    })

info_df = pd.DataFrame(info_records)

print("Computed Information Length for", len(info_df), "reactions")
print(info_df[["L_global","L_local"]].describe())

/srv/conda/envs/notebook/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:1309: RuntimeWarning: divide by zero encountered in divide
  a = -(dx2) / (dx1 * (dx1 + dx2))
/srv/conda/envs/notebook/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:1309: RuntimeWarning: invalid value encountered in divide
  a = -(dx2) / (dx1 * (dx1 + dx2))
/srv/conda/envs/notebook/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:1310: RuntimeWarning: divide by zero encountered in divide
  b = (dx2 - dx1) / (dx1 * dx2)
/srv/conda/envs/notebook/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:1310: RuntimeWarning: invalid value encountered in divide
  b = (dx2 - dx1) / (dx1 * dx2)
/srv/conda/envs/notebook/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:1311: RuntimeWarning: divide by zero encountered in divide
  c = dx1 / (dx2 * (dx1 + dx2))
/srv/conda/envs/notebook/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:1311: RuntimeWarnin

Computed Information Length for 121 reactions
         L_global    L_local
count  112.000000  84.000000
mean     1.130213   1.289823
std      6.174073   7.122824
min      0.246887   0.265802
25%      0.409140   0.395066
50%      0.478080   0.454885
75%      0.564652   0.523875
max     65.785332  65.725569


/srv/conda/envs/notebook/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:1309: RuntimeWarning: divide by zero encountered in divide
  a = -(dx2) / (dx1 * (dx1 + dx2))
/srv/conda/envs/notebook/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:1309: RuntimeWarning: invalid value encountered in divide
  a = -(dx2) / (dx1 * (dx1 + dx2))
/srv/conda/envs/notebook/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:1310: RuntimeWarning: divide by zero encountered in divide
  b = (dx2 - dx1) / (dx1 * dx2)
/srv/conda/envs/notebook/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:1310: RuntimeWarning: invalid value encountered in divide
  b = (dx2 - dx1) / (dx1 * dx2)
/srv/conda/envs/notebook/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:1311: RuntimeWarning: divide by zero encountered in divide
  c = dx1 / (dx2 * (dx1 + dx2))
/srv/conda/envs/notebook/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:1311: RuntimeWarnin

In [24]:
# ============================================
# SECTION 6 — CORRELATION ANALYSIS FOR L
# ============================================

# Merge structure features
info_df = info_df.merge(
    switch_df[["Reaction"]],
    on="Reaction"
)

info_df = info_df.merge(
    df.groupby("Reaction").first().reset_index()[
        ["Reaction","β P","β T","Q ( 2 n )"]
    ],
    on="Reaction"
)

info_df["beta_eff"] = abs(info_df["β P"]) + abs(info_df["β T"])

from scipy.stats import pearsonr

print("Correlation L_global vs beta_eff:")
print(pearsonr(info_df["L_global"], info_df["beta_eff"]))

print("\nCorrelation L_local vs beta_eff:")
print(pearsonr(info_df["L_local"].dropna(),
               info_df.loc[info_df["L_local"].notna(),"beta_eff"]))

print("\nCorrelation L_global vs Q(2n):")
print(pearsonr(info_df["L_global"], info_df["Q ( 2 n )"]))

Correlation L_global vs beta_eff:
PearsonRResult(statistic=np.float64(nan), pvalue=np.float64(nan))

Correlation L_local vs beta_eff:
PearsonRResult(statistic=np.float64(0.24720399700958443), pvalue=np.float64(0.023388891551724602))

Correlation L_global vs Q(2n):
PearsonRResult(statistic=np.float64(nan), pvalue=np.float64(nan))


# Section 5 — Information Length: Physical Interpretation and Assessment

## 5.1 What Information Length Measures

The Information Length (L) is defined as

$$
L = \int \sqrt{\sum_k \left(\frac{d\pi_k}{dE}\right)^2} \, dE,
$$

where $\pi_k(E)$ are the mixture weights predicted by the MDN for each dynamical regime.

In simple terms, $L$ measures how rapidly the system’s probabilistic identity changes as the collision energy increases.

- If one regime smoothly evolves into another, $L$ remains small.
- If the regime probabilities rearrange sharply over a narrow energy window, $L$ becomes large.

Thus, $L$ quantifies the *geometric motion in probability space* as energy increases. It is a measure of transition sharpness, not transition location.

Importantly, once the MDN is trained, the calculation of $L$ does **not** use the original 29 input features. It depends only on the learned regime weights $\pi_k(E)$. The structural features enter only indirectly through how they influenced the trained MDN.

---

## 5.2 Global vs Local Information Length

Two variants were evaluated:

- **Global $L$** — integrated over the full energy range.
- **Local $L$** — integrated within a window around the transition coordinate $x_{\text{switch}}$.

The goal was to determine whether transition sharpness encodes structural or universal behavior similar to the transition location.

### Observations:

1. Global $L$ exhibited large variance and occasional extreme outliers.
2. These outliers arise from the derivative-based nature of $L$, which amplifies small fluctuations in $\pi_k(E)$.
3. Local $L$ showed weak but statistically significant correlation with deformation ($\beta_{\text{eff}}$), with $r \approx 0.25$.

However, the magnitude of this correlation is modest compared to the strong structural scaling observed for $x_{\text{switch}}$ (where $r \approx 0.80$).

---

## 5.3 Interpretation

The results suggest a clear distinction:

- The **transition location** ($x_{\text{switch}}$) is a robust, structure-controlled dynamical threshold.
- The **transition sharpness** ($L$) is weaker and more sensitive to numerical resolution and derivative amplification.

This is physically reasonable.

The energy scale at which the system changes dynamical identity appears to be tightly governed by nuclear structure (deformation and transfer energetics). In contrast, the sharpness of that change depends on finer details such as barrier distribution width, coupling strength variations, and energy sampling density.

Thus, Information Length does not exhibit universality comparable to the transition coordinate itself.

---

## 5.4 Scientific Conclusion on L

Information Length serves as a secondary diagnostic tool that characterizes the sharpness of probabilistic regime evolution. However:

- It does not display strong universal scaling.
- It does not outperform the transition coordinate in structural correlation.
- It is numerically more fragile due to its derivative-based definition.

Therefore, while informative, $L$ does not constitute the primary physical observable of this study.

The central dynamical quantity remains the barrier-normalized transition coordinate:

$$
x_{\text{switch}} = \frac{E_{\text{switch}}}{V_B}.
$$

This coordinate demonstrates both universality and strong structural modulation, whereas Information Length provides only a supplementary characterization of transition sharpness.

# Section 6 — Transition Location vs Transition Sharpness

The study reveals a clear hierarchy between two classes of observables:

| Observable | Physical Meaning | Structural Correlation | Stability |
|------------|------------------|------------------------|-----------|
| $x_{\text{switch}}$ | Energy at which dominant regime changes | Strong ($R^2 \approx 0.75$) | High |
| $L$ | Sharpness of regime rearrangement | Weak | Moderate |

The transition coordinate exhibits:

- Universality across reaction systems.
- Strong correlation with deformation ($\beta_{\text{eff}}$).
- Secondary modulation from $Q_{2n}$.
- Stability across neural network seeds.

In contrast, Information Length:

- Is sensitive to derivative amplification.
- Shows modest structural dependence.
- Does not exhibit clear universal clustering.

This distinction indicates that the primary dynamical signature in heavy-ion fusion is the *location* of the regime transition, rather than the detailed sharpness of that transition.

The probabilistic decomposition therefore reveals a structure-controlled energy threshold governing the onset of the adiabatic/necking regime, while the fine-grained transition dynamics remain system-specific.

In summary:

The universality resides in *where* the transition occurs, not in *how sharply* it occurs.

In [28]:
# ============================================
# SECTION 7 — FULL 29-FEATURE REGRESSION
# ============================================

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import numpy as np

# ---------------------------------------
# Prepare reliable dataset
# ---------------------------------------

reliable_reactions = switch_df[switch_df["reliable"]]["Reaction"]

df_struct = (
    df.groupby("Reaction")
      .first()
      .reset_index()
)

df_struct = df_struct[df_struct["Reaction"].isin(reliable_reactions)]

df_struct = df_struct.merge(
    switch_df[["Reaction","x_switch_mean"]],
    on="Reaction"
)

# 29 features
features_all = features_train.copy()

X = df_struct[features_all].values
y = df_struct["x_switch_mean"].values

# Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ---------------------------------------
# Cross-validated Linear Regression
# ---------------------------------------

model = LinearRegression()

kf = KFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = cross_val_score(model, X_scaled, y,
                            cv=kf, scoring='r2')

print("Cross-validated R² scores:", cv_scores)
print("Mean CV R²:", np.mean(cv_scores))

# Fit full model for inspection
model.fit(X_scaled, y)
print("Full-fit R²:", r2_score(y, model.predict(X_scaled)))

Cross-validated R² scores: [0.77418409 0.67984962 0.72126519 0.62132631 0.59076587]
Mean CV R²: 0.6774782153859705
Full-fit R²: 0.880803028404844


In [29]:
# ============================================
# SHUFFLED BASELINE TEST
# ============================================

n_shuffle = 200
shuffle_scores = []

for i in range(n_shuffle):
    y_shuffled = np.random.permutation(y)
    score = np.mean(
        cross_val_score(model, X_scaled, y_shuffled,
                        cv=kf, scoring='r2')
    )
    shuffle_scores.append(score)

print("Mean shuffled R²:", np.mean(shuffle_scores))
print("Max shuffled R²:", np.max(shuffle_scores))

Mean shuffled R²: -0.5163073401454418
Max shuffled R²: -0.16219976927155072


In [30]:
# ============================================
# SECTION 8 — FEATURE SELECTION (LASSO)
# ============================================

from sklearn.linear_model import LassoCV
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# ---------------------------------------
# Prepare reliable dataset
# ---------------------------------------

reliable_reactions = switch_df[switch_df["reliable"]]["Reaction"]

df_struct = (
    df.groupby("Reaction")
      .first()
      .reset_index()
)

df_struct = df_struct[df_struct["Reaction"].isin(reliable_reactions)]

df_struct = df_struct.merge(
    switch_df[["Reaction","x_switch_mean"]],
    on="Reaction"
)

features_all = features_train.copy()

X = df_struct[features_all].values
y = df_struct["x_switch_mean"].values

# Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ---------------------------------------
# LASSO with cross-validation
# ---------------------------------------

lasso = LassoCV(cv=5, random_state=42)
lasso.fit(X_scaled, y)

print("Optimal alpha:", lasso.alpha_)
print("LASSO R²:", lasso.score(X_scaled, y))

# ---------------------------------------
# Extract selected features
# ---------------------------------------

coef = pd.Series(lasso.coef_, index=features_all)

selected_features = coef[coef != 0].sort_values(ascending=False)

print("\nSelected Features:")
print(selected_features)

Optimal alpha: 0.0003319814726235651
LASSO R²: 0.8417759471186921

Selected Features:
R B                          0.036134
Compound_Nucleus_S2n         0.008926
magic_dist_N1                0.005274
magic_dist_N2                0.002350
N1                           0.000242
Compound_Nucleus_Sn         -0.000951
β T                         -0.001117
Compound_Nucleus_Sp         -0.001790
magic_dist_Z1               -0.003206
Projectile_Binding_Energy   -0.007158
β P                         -0.007478
Q ( 2 n )                   -0.008061
Target_Binding_Energy       -0.009533
E c.m.                      -0.011745
Z1Z2_over_Ecm               -0.016446
ħ ω                         -0.021254
dtype: float64


/srv/conda/envs/notebook/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:701: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.167e-05, tolerance: 1.391e-05
  model = cd_fast.enet_coordinate_descent_gram(
/srv/conda/envs/notebook/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:701: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.266e-05, tolerance: 1.391e-05
  model = cd_fast.enet_coordinate_descent_gram(
/srv/conda/envs/notebook/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:701: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regul

In [31]:
# ============================================
# SECTION 9 — CLEAN REACTION-LEVEL LASSO
# ============================================

from sklearn.linear_model import LassoCV
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# ---------------------------------------
# Define intrinsic reaction-level features
# ---------------------------------------

reaction_features = [
    'Z1','N1','A1',
    'Z2','N2','A2',
    'Q ( 2 n )',
    'magic_dist_Z1','magic_dist_N1',
    'magic_dist_Z2','magic_dist_N2',
    'Z3','N3','A3',
    'β P','β T',
    'R B','ħ ω',
    'Projectile_Mass_Actual',
    'Target_Mass_Actual',
    'Compound_Nucleus_Mass_Actual',
    'Compound_Nucleus_Sp',
    'Compound_Nucleus_Sn',
    'Projectile_Binding_Energy',
    'Target_Binding_Energy',
    'Compound_Nucleus_Binding_Energy',
    'Compound_Nucleus_S2n'
]

# ---------------------------------------
# Prepare dataset
# ---------------------------------------

reliable_reactions = switch_df[switch_df["reliable"]]["Reaction"]

df_struct = (
    df.groupby("Reaction")
      .first()
      .reset_index()
)

df_struct = df_struct[df_struct["Reaction"].isin(reliable_reactions)]

df_struct = df_struct.merge(
    switch_df[["Reaction","x_switch_mean"]],
    on="Reaction"
)

X = df_struct[reaction_features].values
y = df_struct["x_switch_mean"].values

# Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ---------------------------------------
# LASSO with cross-validation
# ---------------------------------------

lasso = LassoCV(cv=5, random_state=42)
lasso.fit(X_scaled, y)

print("Optimal alpha:", lasso.alpha_)
print("Training R²:", lasso.score(X_scaled, y))

# ---------------------------------------
# Cross-validated R²
# ---------------------------------------

kf = KFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = cross_val_score(lasso, X_scaled, y,
                            cv=kf, scoring='r2')

print("Cross-validated R² scores:", cv_scores)
print("Mean CV R²:", np.mean(cv_scores))

# ---------------------------------------
# Selected features
# ---------------------------------------

coef = pd.Series(lasso.coef_, index=reaction_features)

selected_features = coef[coef != 0].sort_values(ascending=False)

print("\nSelected Features:")
print(selected_features)

/srv/conda/envs/notebook/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:701: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.520e-05, tolerance: 1.497e-05
  model = cd_fast.enet_coordinate_descent_gram(
/srv/conda/envs/notebook/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:701: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.027e-05, tolerance: 1.497e-05
  model = cd_fast.enet_coordinate_descent_gram(
/srv/conda/envs/notebook/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:701: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regul

Optimal alpha: 0.00010138234490021039
Training R²: 0.8262069085026964


/srv/conda/envs/notebook/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:701: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.374e-05, tolerance: 1.111e-05
  model = cd_fast.enet_coordinate_descent_gram(
/srv/conda/envs/notebook/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:701: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.387e-05, tolerance: 1.111e-05
  model = cd_fast.enet_coordinate_descent_gram(
/srv/conda/envs/notebook/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:716: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regul

Cross-validated R² scores: [0.69269552 0.07068935 0.61549    0.68036163 0.74896735]
Mean CV R²: 0.5616407696236794

Selected Features:
R B                                0.039115
Compound_Nucleus_Binding_Energy    0.011086
Compound_Nucleus_S2n               0.006666
magic_dist_N1                      0.004638
N1                                 0.002088
magic_dist_N2                      0.001582
magic_dist_Z2                     -0.000896
Compound_Nucleus_Sn               -0.003050
β T                               -0.003065
magic_dist_Z1                     -0.003652
Compound_Nucleus_Sp               -0.006520
Projectile_Binding_Energy         -0.009365
β P                               -0.009385
ħ ω                               -0.010240
Q ( 2 n )                         -0.011616
Target_Binding_Energy             -0.020226
Z3                                -0.033570
dtype: float64


/srv/conda/envs/notebook/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:701: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.802e-05, tolerance: 1.353e-05
  model = cd_fast.enet_coordinate_descent_gram(
/srv/conda/envs/notebook/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:701: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.203e-05, tolerance: 1.353e-05
  model = cd_fast.enet_coordinate_descent_gram(
/srv/conda/envs/notebook/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:701: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regul

In [32]:
# ============================================
# SECTION 10 — NONLINEAR STRUCTURE MODEL
# ============================================

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score
import numpy as np

# ---------------------------------------
# Use only the two key structural variables
# ---------------------------------------

df_nonlin = df_struct.copy()

df_nonlin["beta_eff"] = abs(df_nonlin["β P"]) + abs(df_nonlin["β T"])

X_simple = df_nonlin[["beta_eff","Q ( 2 n )"]].values
y = df_nonlin["x_switch_mean"].values

# ---------------------------------------
# Polynomial expansion (degree 2)
# ---------------------------------------

poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_simple)

model = LinearRegression()

kf = KFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = cross_val_score(model, X_poly, y,
                            cv=kf, scoring='r2')

print("Polynomial features:", poly.get_feature_names_out())
print("Cross-validated R² scores:", cv_scores)
print("Mean CV R²:", np.mean(cv_scores))

# Fit full model to inspect coefficients
model.fit(X_poly, y)

coef_names = poly.get_feature_names_out()
coef_values = model.coef_

print("\nCoefficients:")
for name, coef in zip(coef_names, coef_values):
    print(f"{name}: {coef}")

Polynomial features: ['x0' 'x1' 'x0^2' 'x0 x1' 'x1^2']
Cross-validated R² scores: [0.61776405 0.74921972 0.69886118 0.70013193 0.75914016]
Mean CV R²: 0.7050234100763637

Coefficients:
x0: 0.18748926805207713
x1: -0.0021605062250205902
x0^2: -0.042282014755786704
x0 x1: -0.004616388964626581
x1^2: -0.000132266328297357


In [33]:
# ============================================
# SECTION 11 — RANDOM FOREST TEST
# ============================================

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score
import numpy as np

# Use intrinsic reaction-level features
X_rf = df_struct[reaction_features].values
y_rf = df_struct["x_switch_mean"].values

rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=5,
    random_state=42
)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = cross_val_score(rf, X_rf, y_rf,
                            cv=kf, scoring='r2')

print("Random Forest CV R² scores:", cv_scores)
print("Mean CV R²:", np.mean(cv_scores))

# Fit to inspect feature importance
rf.fit(X_rf, y_rf)

importances = rf.feature_importances_

feature_importance_df = (
    pd.Series(importances, index=reaction_features)
      .sort_values(ascending=False)
)

print("\nTop 10 Feature Importances:")
print(feature_importance_df.head(10))

Random Forest CV R² scores: [0.51809358 0.61641762 0.62879821 0.79974014 0.77443838]
Mean CV R²: 0.6674975862266571

Top 10 Feature Importances:
β T                             0.188069
β P                             0.120760
R B                             0.095398
Q ( 2 n )                       0.062649
Target_Binding_Energy           0.053697
N3                              0.045688
Projectile_Binding_Energy       0.043073
A3                              0.042388
Compound_Nucleus_Mass_Actual    0.040784
Compound_Nucleus_S2n            0.030178
dtype: float64


In [35]:
!pip install xgboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.7/131.7 MB 142.7 MB/s  0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.8/289.8 MB 207.2 MB/s  0:00:010:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [xgboost]m1/2 [xgboost]


In [36]:
# ============================================
# SECTION 12 — XGBOOST TEST
# ============================================

from xgboost import XGBRegressor
from sklearn.model_selection import KFold, cross_val_score
import numpy as np

# Use intrinsic reaction-level features
X_xgb = df_struct[reaction_features].values
y_xgb = df_struct["x_switch_mean"].values

xgb = XGBRegressor(
    n_estimators=500,
    max_depth=3,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = cross_val_score(
    xgb, X_xgb, y_xgb,
    cv=kf,
    scoring='r2'
)

print("XGBoost CV R² scores:", cv_scores)
print("Mean CV R²:", np.mean(cv_scores))

# Fit to inspect feature importance
xgb.fit(X_xgb, y_xgb)

importances = xgb.feature_importances_

feature_importance_df = (
    pd.Series(importances, index=reaction_features)
      .sort_values(ascending=False)
)

print("\nTop 10 Feature Importances:")
print(feature_importance_df.head(10))

XGBoost CV R² scores: [0.6713767  0.71557127 0.73991773 0.6216439  0.85007044]
Mean CV R²: 0.7197160068715613

Top 10 Feature Importances:
β T                             0.140280
β P                             0.105489
Projectile_Binding_Energy       0.092814
Z2                              0.078512
Z3                              0.069432
A3                              0.055876
N3                              0.046340
R B                             0.044197
Compound_Nucleus_Mass_Actual    0.040386
Z1                              0.039712
dtype: float32


# Section 8 — Sparse Feature Selection (LASSO)

To identify the minimal subset of intrinsic reaction-level features governing the transition coordinate, LASSO regression with 5-fold cross-validation was performed.

Only intrinsic nuclear properties were retained (energy-dependent features removed).

**Results:**

- Mean cross-validated R² ≈ 0.56  

The reduction in performance compared to the deformation + Q₂n model indicates that expanding the feature space does not enhance predictive power. Instead, the transition coordinate appears to be primarily controlled by a small subset of physically meaningful structural variables.

LASSO consistently retained:

- Barrier radius (R_B)
- Deformation parameters (βP, βT)
- Two-neutron transfer Q-value
- Barrier curvature (ħω)
- Selected separation and binding energies

However, the overall predictive performance did not exceed that of the simple structural model, reinforcing the conclusion that the transition coordinate is fundamentally low-dimensional.
    # Section 8 — Sparse Feature Selection (LASSO)

To identify the minimal subset of intrinsic reaction-level features governing the transition coordinate, LASSO regression with 5-fold cross-validation was performed.

Only intrinsic nuclear properties were retained (energy-dependent features removed).

**Results:**

- Mean cross-validated R² ≈ 0.56  

The reduction in performance compared to the deformation + Q₂n model indicates that expanding the feature space does not enhance predictive power. Instead, the transition coordinate appears to be primarily controlled by a small subset of physically meaningful structural variables.

LASSO consistently retained:

- Barrier radius (R_B)
- Deformation parameters (βP, βT)
- Two-neutron transfer Q-value
- Barrier curvature (ħω)
- Selected separation and binding energies

However, the overall predictive performance did not exceed that of the simple structural model, reinforcing the conclusion that the transition coordinate is fundamentally low-dimensional.

# Section 10 — Nonlinear Structural Model

To test whether nonlinear interactions between deformation and transfer energetics improve explanatory power, a quadratic polynomial expansion was constructed:

x_switch = f(β_eff, Q₂n, β_eff², β_eff·Q₂n, Q₂n²)

Using 5-fold cross-validation:

- Mean CV R² ≈ 0.70  

The performance does not exceed that of the linear deformation + Q₂n model.

The quadratic coefficient in β_eff is small and negative, indicating weak curvature but no strong nonlinear enhancement.

This suggests that the structural dependence of the transition coordinate is approximately linear in deformation, with only minor higher-order corrections.
    # Section 11 — Random Forest Stress Test

A Random Forest regressor was applied to the intrinsic reaction-level features to test for hidden nonlinear structure.

5-fold cross-validation yielded:

- Mean CV R² ≈ 0.67  

Feature importance rankings identified deformation parameters (βP, βT) as the dominant contributors, followed by barrier geometry (R_B) and transfer energetics (Q₂n).

The performance does not exceed that of the simple linear structural model.

This indicates that nonlinear ensemble methods do not uncover additional predictive structure beyond deformation and transfer effects.
# Section 12 — XGBoost Boosted Model Test

To further test for subtle nonlinear interactions, a gradient-boosted decision tree model (XGBoost) was evaluated.

5-fold cross-validation yielded:

- Mean CV R² ≈ 0.72  

Feature importance again ranked deformation parameters as dominant, followed by selected binding and mass terms.

Although XGBoost captures nonlinear structure more effectively than Random Forest, it does not outperform the simple linear deformation + Q₂n model.

This confirms that the transition coordinate is not a high-dimensional nonlinear artifact, but instead reflects a fundamentally low-dimensional structural scaling law.
# Section 13 — Model Comparison and Structural Conclusion

A systematic comparison of all tested models reveals:

| Model | Mean CV R² |
|--------|------------|
| Linear (β_eff + Q₂n) | ~0.75 |
| Quadratic (β_eff, Q₂n) | ~0.70 |
| Full 29-feature linear | ~0.68 |
| Random Forest | ~0.67 |
| XGBoost | ~0.72 |
| Clean LASSO | ~0.56 |

The simple linear structural model consistently demonstrates the strongest generalization performance.

This indicates that:

1. The transition coordinate is primarily governed by deformation.
2. Two-neutron transfer energetics provide secondary modulation.
3. Barrier geometry plays a supporting role.
4. High-dimensional nonlinear ML models do not uncover additional hidden structure.

Therefore, the barrier-normalized transition coordinate can be expressed approximately as:

x_switch ≈ 0.88 + 0.16 β_eff − 0.003 Q₂n

This result demonstrates that the emergent transition scale discovered via probabilistic regime decomposition collapses onto a compact, interpretable nuclear structure law.

In [38]:
!pip install pysr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 209.8 MB/s  0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 3.0.1
    Uninstalling pandas-3.0.1:╺━━━━━━━━━━━━━━━━━━━━━━ 3/7 [pandas]
      Successfully uninstalled pandas-3.0.1m━━━━━━━━━━━━━━━━━━━━━━ 3/7 [pandas]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [pysr]6/7 [pysr]s]


In [39]:
import pysr

[juliapkg] Found dependencies: /srv/conda/envs/notebook/lib/python3.11/site-packages/pysr/juliapkg.json
[juliapkg] Found dependencies: /srv/conda/envs/notebook/lib/python3.11/site-packages/awkward/juliapkg.json
[juliapkg] Found dependencies: /srv/conda/envs/notebook/lib/python3.11/site-packages/juliapkg/juliapkg.json
[juliapkg] Found dependencies: /srv/conda/envs/notebook/lib/python3.11/site-packages/juliacall/juliapkg.json
[juliapkg] Locating Julia ^1.10.3
[juliapkg] Querying Julia versions from https://julialang-s3.julialang.org/bin/versions.json
[juliapkg] WARNING: About to install Julia 1.12.5 to /srv/conda/envs/notebook/julia_env/pyjuliapkg/install.
[juliapkg]   If you use juliapkg in more than one environment, you are likely to
[juliapkg]   have Julia installed in multiple locations. It is recommended to
[juliapkg]   install JuliaUp (https://github.com/JuliaLang/juliaup) or Julia
[juliapkg]   (https://julialang.org/downloads) yourself.
[juliapkg] Downloading Julia from https://ju

  Installing known registries into `~/.julia`
       Added `General` registry to ~/.julia/registries
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed Tricks ────────────────────── v0.1.13
   Installed ScientificTypesBase ───────── v3.1.0
   Installed IrrationalConstants ───────── v0.2.6
   Installed Adapt ─────────────────────── v4.4.0
   Installed DiffRules ─────────────────── v1.15.1
   Installed Scratch ───────────────────── v1.3.0
   Installed DynamicExpressions ────────── v1.10.3
   Installed MicroMamba ────────────────── v0.1.15
   Installed Conda ─────────────────────── v1.10.3
   Installed JSON ──────────────────────── v0.21.4
   Installed MLJModelInterface ─────────── v1.11.1
   Installed JSON3 ─────────────────────── v1.14.3
   Installed TableTraits ───────────────── v1.0.1
   Installed PythonCall ────────────────── v0.9.26
   Installed PositiveFactorizations ────── v0.2.4
   Installed ADTypes ───────────────────── v1.2

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [45]:
features_sr = [
    "beta_eff",
    "Q ( 2 n )"
]

sr_df = switch_df[switch_df["reliable"] == True].copy()

X_sr = sr_df[features_sr].values
y_sr = sr_df["x_switch_mean"].values

print("Dataset size:", X_sr.shape)

Dataset size: (137, 2)


In [ ]:
from pysr import PySRRegressor
import multiprocessing

n_cores = multiprocessing.cpu_count()
print("Detected CPU cores:", n_cores)

model = PySRRegressor(
    niterations=2000,              # we don't need 40k
    populations=40,
    population_size=1000,
    ncycles_per_iteration=500,
    binary_operators=["+", "-", "*", "/"],
    unary_operators=["square"],
    maxsize=15,
    model_selection="best",
    loss="loss(x, y) = (x - y)^2",
    procs=n_cores,                 # THIS is critical
    progress=True,
    verbosity=1,
)

model.fit(X_sr, y_sr)

In [42]:
# Build one row per reaction with structural features

structure_df = df.groupby("Reaction").first().reset_index()

structure_df["beta_eff"] = (
    abs(structure_df["β P"]) + abs(structure_df["β T"])
)

structure_df = structure_df[[
    "Reaction",
    "beta_eff",
    "Q ( 2 n )"
]]

In [43]:
switch_df = switch_df.merge(
    structure_df,
    on="Reaction",
    how="left"
)

print(switch_df[["Reaction", "beta_eff", "Q ( 2 n )"]].head())

        Reaction  beta_eff  Q ( 2 n )
0    12 C + 89 Y     0.355      -7.71
1   12 C + 92 Zr     0.373      -2.71
2  12 C + 144 Sm     0.320      -6.00
3  12 C + 152 Sm     0.563      -0.73
4  12 C + 154 Sm     0.590      -0.71


In [44]:
features_sr = [
    "beta_eff",
    "Q ( 2 n )"
]

sr_df = switch_df[switch_df["reliable"] == True].copy()

X_sr = sr_df[features_sr].values
y_sr = sr_df["x_switch_mean"].values

print("Dataset size:", X_sr.shape)

Dataset size: (137, 2)
